In [11]:
import pandas as pd
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
from scipy.stats import gaussian_kde
from scipy.stats import entropy
import geopandas as gpd
# import matplotlib.pyplot as plt
import pandas as pd

from matplotlib.backends.backend_pdf import PdfPages
import sys
sys.path.append('/project/lhansen/HMC_re/project-amazon/')
from pysrc.sampling import gibbs_sampling

# import pathlib
# temp = pathlib.PosixPath
# pathlib.PosixPath = pathlib.WindowsPath


    
site=78



base_folder = "/project/lhansen/HMC_re/project-amazon/output/sampling/gams/78sites/pa_41.11/"

with open(base_folder+'xi_1/pe_5.3/results.pcl', 'rb') as f:
    # Load the data from the file
    results_adjusted = pickle.load(f)    
beta_hmc =results_adjusted["final_sample_coe"][:,:8]
v_hmc = results_adjusted["V_gamma_sample"]

In [12]:

res = gibbs_sampling("gamma")
gamma_mean = res['beta_gamma_mean']
gamma_vcov = res['beta_gamma_vcov']
beta_ori = np.random.multivariate_normal(gamma_mean,gamma_vcov,size=50000)
v_mean = res['V_gamma_mean']
v_var   = res['V_gamma_var']
v_ori = np.zeros((50000,78))

for i in range(78):
    v_ori[:,i] = np.random.normal(v_mean[i],np.sqrt(v_var[i]),size=50000)

In [13]:
gamma_data = gpd.read_file("/project/lhansen/HMC_re/project-amazon/data/calibration/hmc/"+"gamma_data_site_78.geojson")
X_fit = gamma_data.iloc[:, 0:6].values  # Columns 1 to 6 as X
id_fit = gamma_data.iloc[:, 7].values  # Columns 1 to 6 as X

gamma_baseline= np.exp(X_fit @ beta_ori.T + v_ori[:,].T).T
gamma_hmc=results_adjusted["final_sample"][:, 78:] 

In [ ]:


for i in range(78):
    plt.figure(figsize=(12, 6))
    sns.kdeplot(gamma_baseline[:, i], label=r'baseline', color='red', fill=True, alpha=0.6,linewidth=4) 
    sns.kdeplot(gamma_hmc[:, i], label=r'hmc', color='blue', fill=True, alpha=0.6,linewidth=4)  
    # sns.kdeplot(gamma_test[:, i], label=r'test', color='blue', fill=True, alpha=0.6,linewidth=4)  
    plt.title(r'Density of $\gamma_'+str(i+1)+'$')
    plt.legend()
    # plt.savefig('results/compare/gamma'+str(i+1)+'.png')
    plt.show()


In [15]:
mean_values = np.mean(gamma_hmc, axis=0)
df_mean = pd.DataFrame(mean_values, index=pd.Index(range(1, len(mean_values) + 1), name="Site"))
df_mean.to_csv('mean_post_adj.csv', index=True)

In [16]:
sd_hmc=np.sqrt(np.var(gamma_hmc,ddof=0,axis=0))
quantiles = np.quantile(gamma_hmc, [0.1, 0.5, 0.9], axis=0).T
pd.DataFrame(quantiles).to_csv('quantiles.csv')

In [ ]:
from scipy.stats import gaussian_kde
from scipy.stats import entropy
kl_divergences_gamma = []

for idx in range(site):


    
    kde_func_unadjusted = gaussian_kde(gamma_baseline[:, idx], bw_method='scott')
    kde_func_adjusted = gaussian_kde(gamma_hmc[:, idx], bw_method='scott')

    common_grid = np.linspace(min(gamma_baseline[:, idx].min(), gamma_hmc[:, idx].min()), 
                            max(gamma_baseline[:, idx].max(), gamma_hmc[:, idx].max()), 
                            1000)
    density_unadjusted = kde_func_unadjusted(common_grid)
    density_adjusted = kde_func_adjusted(common_grid)
    density_unadjusted += 1e-20
    density_adjusted += 1e-20
    
    kl_div = entropy(density_unadjusted, density_adjusted)
    print(f'Site {idx+1} KL Divergence: {kl_div}')

    kl_divergences_gamma.append({'Site': idx+1, 'KL_Divergence': kl_div})

kl_df_gamma = pd.DataFrame(kl_divergences_gamma)

# Save the DataFrame to a CSV file
kl_df_gamma.to_csv('kl_gamma.csv', index=False)